# Final Project: Decide on global dual sourcing strategy

team: \
David Yang \
Jack Chen \
Joyce Wu

In [107]:
import numpy as np
import pandas as pd

In [108]:
data = pd.read_csv('final project 2024.csv')

In [109]:
data.head(10)

,period,demand
0,1,21
1,2,81
2,3,32
3,4,58
4,5,47
5,6,49
6,7,66
7,8,29
8,9,55
9,10,39


In [110]:
s = 10 # sales price
c_m = 8 # cost mexico
c_c = 7.25 # cost china
i = 0.01 # interest rate
ini_bal = 0 # initial balance

## Duo Sourcing

In [111]:
Days_loop = np.arange(2, 9)

In [112]:
emergency = np.arange(0.0, 0.5, 0.025)

### Cumulative Average Forecast

In [113]:
data

,period,demand
0,1,21
1,2,81
2,3,32
3,4,58
4,5,47
...,...,...
9995,9996,68
9996,9997,70
9997,9998,74
9998,9999,52


In [114]:
def China_cul_avg_n_days(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, days, er_perc):
    cash_balance = initial_cash_balance
    inventory = 0
    orders = []
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]
        
        # Calculate cumulative average demand up to the current period
        if period > 1:
            cum_avg_demand = np.ceil(np.mean(data.loc[data['period'] <= period, 'demand']))
        else:
            cum_avg_demand = np.ceil(current_demand)  # For the first period, use the current demand

        # Check if any orders are arriving in this period
        orders_to_receive = [order for order in orders if order[0] == period]
        for order in orders_to_receive:
            inventory += order[1]

        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0
        
        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue

        # Determine order quantity based
        if cum_avg_demand*days >= inventory:
            if inventory < er_perc * cum_avg_demand*days:
                sourcing_cost = 8
                order_quantity = int(cum_avg_demand - inventory)
                orders.append((period+1, order_quantity))
            else:
                sourcing_cost = 7.25
                order_quantity = int(cum_avg_demand*days - inventory)
                orders.append((period+4, order_quantity))
        else:
            order_quantity = 0

        order_cost = order_quantity * sourcing_cost
        cash_balance -= order_cost
    
        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

        # Update inventory with new order
        inventory += order_quantity
    
    return cash_balance



In [115]:
max_balance = 0
max_er = 0
max_day = 0

for er in emergency:
    print(f'er {er}')
    for day in Days_loop:
        balance = China_cul_avg_n_days(data, s, c_c, i, ini_bal, day, er)
        if balance > max_balance:
            max_balance = balance
            max_day = day
            max_er = er
print(f'Current er{max_er}, Day {max_day}, Total Balanced: {max_balance}')

KeyboardInterrupt: 

In [ ]:
max_balance_model_1 = max_balance
max_day_model_1 = max_er

max_balance_model_1, max_day_model_1

(1.5276923510596473e+47, 0.3)

In [99]:
def China_moving_avg_n_days(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, moving_avg_window, days, er_perc):
    cash_balance = initial_cash_balance
    inventory = 0
    orders= []
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]
        # demand_lst.append(current_demand)
        
        # Calculate cumulative average demand up to the current period
        if period >= moving_avg_window:
            moving_avg_demand = np.ceil(np.mean(data.loc[(data['period'] > period - moving_avg_window) & (data['period'] <= period), 'demand']))
        else:
            moving_avg_demand = np.ceil(np.mean(data.loc[data['period'] <= period, 'demand']))
        
        orders_to_receive = [order for order in orders if order[0] == period]
        for order in orders_to_receive:
            inventory += order[1]

        avg_lead_demand =  np.ceil(np.mean(data.loc[data['period'] <= period, 'demand']))
        
        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0
        
        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue

        # Determine order quantity based
        if avg_lead_demand*days >= inventory:
            if inventory < er_perc * avg_lead_demand*days:
                sourcing_cost = 8
                order_quantity = int(moving_avg_demand - inventory)
                orders.append((period+1, order_quantity))
            else:
                sourcing_cost = 7.25
                order_quantity = int(moving_avg_demand*days - inventory)
                orders.append((period+4, order_quantity))
        else:
            order_quantity = 0
        
        order_cost = order_quantity * sourcing_cost
        cash_balance -= order_cost

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

        # Update inventory with new order
        inventory += order_quantity
    
    return cash_balance



In [100]:
max_balance = 0
max_er = 0
max_day = 0

for day in Days_loop:
    print(f'day {day}')
    for er in emergency:
        balance = China_moving_avg_n_days(data, s, c_c, i, ini_bal, 10, day, er)
        if balance > max_balance:
            max_balance = balance
            max_er = er
            max_day = day
print(f'Current er{max_er}, Days {max_day}, Total Balanced: {max_balance}')

Current er0.3, Days 2, Total Balanced: 1.3196142962122923e+47


In [101]:
max_balance_model_2 = max_balance
max_day_model_2 = max_er

max_balance_model_2, max_day_model_2

(1.3196142962122923e+47, 0.3)

### Exponential Smoothing

In [102]:
def China_exp_n_days(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, alpha, days, er_perc):
    cash_balance = initial_cash_balance
    inventory = 0
    orders = []
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]
        
        # Calculate cumulative average demand up to the current period
        if period == 1:
            forecast = current_demand  # Use the first period's demand as the initial forecast
        else:
            forecast = alpha * current_demand + (1 - alpha) * forecast
        avg_lead_demand =  np.ceil(np.mean(data.loc[data['period'] <= period, 'demand']))

        orders_to_receive = [order for order in orders if order[0] == period]
        for order in orders_to_receive:
            inventory += order[1]
        
        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0
        
        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue

        # Determine order quantity based
        if avg_lead_demand*days >= inventory:
            if inventory < er_perc * avg_lead_demand*days:
                sourcing_cost = 8
                order_quantity = int(forecast - inventory)
                orders.append((period+1, order_quantity))
            else:
                sourcing_cost = 7.25
                order_quantity = int(forecast*days - inventory)
                orders.append((period+4, order_quantity))
        else:
            order_quantity = 0
        
        order_cost = order_quantity * sourcing_cost
        cash_balance -= order_cost

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

        # Update inventory with new order
        inventory += order_quantity
    
    return cash_balance



In [103]:
# alphas = np.arange(0.01, 1, 0.01)

In [104]:
max_balance = 0
max_er = 0
max_day = 0

for day in Days_loop:
    print(f'day {day}')
    for er in emergency:
        balance = China_exp_n_days(data, s, c_c, i, ini_bal, 0.1, day, er)
        if balance > max_balance:
            max_balance = balance
            max_er = er
            max_day = day

print(f'Max er {max_er}, Max day {max_day}, Total Balanced: {max_balance}')

Max er 0.3, Max day 2, Total Balanced: 1.2889829443750143e+47


In [105]:
max_balance_model_3 = max_balance
max_er_model_3 = max_er

max_balance_model_3, max_er_model_3

(1.2889829443750143e+47, 0.3)

### Results

In [106]:
print("Cumulative Average Forecast: ${:,.2e}".format(max_balance_model_1))
print("10-Day Moving Average Forecast: ${:,.2e}".format(max_balance_model_2))
print(f"Exponential Smoothing Forecast: ${max_balance_model_3:,.2e}")

Cumulative Average Forecast: $1.53e+47
10-Day Moving Average Forecast: $1.32e+47
Exponential Smoothing Forecast: $1.29e+47
